<a href="https://colab.research.google.com/github/AndersonxxAlves/Projeto-01/blob/main/agente_virtual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
import streamlit as st
import json
from pathlib import Path
import requests

class BancoDados:
    def __init__(self, pasta_dados="."):
        self.pasta = Path(pasta_dados)
        self._carregar_dados()

    # =========== CARREGAMENTO ==============
    def _carregar_dados(self):
        self.saldos = pd.read_csv(self.pasta / "saldos_bancarios_ficticios.csv")
        self.extrato = pd.read_csv(self.pasta / "extrato_bancario_ficticio.csv")
        self.limites = pd.read_csv(self.pasta / "limites_cartoes_ficticios.csv")

        with open(self.pasta / "emprestimos_ficticios.json", encoding="utf-8") as f:
            self.emprestimos = pd.DataFrame(json.load(f))

        with open(self.pasta / "bloqueio_cartoes_ficticios.json", encoding="utf-8") as f:
            self.bloqueios = pd.DataFrame(json.load(f))

        with open(self.pasta / "segunda_via_boletos_ficticios.json", encoding="utf-8") as f:
            self.boletos = pd.DataFrame(json.load(f))

    # ========= CONSULTAS PRINCIPAIS ============
    def obter_saldo(self, cliente_id):
        return self.saldos[self.saldos["cliente_id"] == cliente_id]

    def obter_extrato(self, cliente_id):
        return self.extrato[self.extrato["cliente_id"] == cliente_id]

    def obter_limites_cartao(self, cliente_id):
        return self.limites[self.limites["cliente_id"] == cliente_id]

    def obter_emprestimos(self, cliente_id):
        return self.emprestimos[self.emprestimos["cliente_id"] == cliente_id]

    def obter_bloqueios_cartao(self, cliente_id):
        return self.bloqueios[self.bloqueios["cliente_id"] == cliente_id]

    def obter_boletos(self, cliente_id):
        return self.boletos[self.boletos["cliente_id"] == cliente_id]

In [23]:
class AssistenteBancario:
    def __init__(self, banco_dados):
        self.db = banco_dados

    def consultar_saldo(self, cliente_id):
        dados = self.db.obter_saldo(cliente_id)
        if dados.empty:
            return "Cliente não encontrado."
        saldo = dados.iloc[0]["saldo"]
        return f"Seu saldo atual é R$ {saldo:,.2f}"

    def consultar_limite_cartao(self, cliente_id):
        dados = self.db.obter_limites_cartao(cliente_id)
        if dados.empty:
            return "Limites não encontrados."
        disp = dados.iloc[0]["cartao_credito_disponivel"]
        return f"Seu limite disponível é R$ {disp:,.2f}"

    def consultar_status_bloqueio(self, cliente_id):
        dados = self.db.obter_bloqueios_cartao(cliente_id)
        if dados.empty:
            return "Nenhum bloqueio encontrado."
        status = dados.iloc[0]["status_bloqueio"]
        return f"Status do bloqueio do cartão: {status}"

    def gerar_contexto_cliente(self, cliente_id):
        saldo = self.db.obter_saldo(cliente_id)
        limite = self.db.obter_limites_cartao(cliente_id)
        bloqueio = self.db.obter_bloqueios_cartao(cliente_id)

        contexto = f"""
        Cliente ID: {cliente_id}

        SALDO:
        {saldo.to_dict(orient="records")}

        LIMITE CARTÃO:
        {limite.to_dict(orient="records")}

        BLOQUEIO:
        {bloqueio.to_dict(orient="records")}
        """
        return contexto

In [24]:
# =========== SYSTEM PROMPT ===========

SYSTEM_PROMPT = """ Íris é uma agente virtual de assistência bancária criada para tornar a experiência financeira dos clientes mais simples, rápida e segura. Com uma comunicação clara, amigável e objetiva, ela entende as necessidades do usuário e oferece suporte imediato no dia a dia, ajudando a acompanhar informações importantes da conta e a resolver demandas comuns sem complicação.

Objetivo:
A Íris tem como objetivo principal oferecer ao cliente uma experiência bancária ágil, segura e autônoma para o acompanhamento e a gestão de suas principais demandas financeiras do dia a dia. Sua proposta é centralizar, em um único canal de atendimento, as interações mais recorrentes relacionadas à consulta de saldo, visualização de extrato, simulação de empréstimo, verificação de limite do cartão, emissão de segunda via de boletos e bloqueio de cartão.

REGRAS:
1 - Não realizar transações financeiras
Íris não deve efetuar transferências, pagamentos, investimentos ou qualquer movimentação de valores.

2 - Não alterar dados cadastrais do cliente
A agente não pode modificar informações pessoais, como endereço, telefone, e-mail ou dados bancários.

3 - Não fornecer aconselhamento financeiro personalizado
Íris não deve recomendar produtos financeiros, investimentos ou decisões de crédito com caráter consultivo.

4 - Não aprovar ou contratar produtos
A agente não tem permissão para efetivar contratação de empréstimos, cartões ou quaisquer serviços — apenas apresentar simulações informativas.

5 - Não compartilhar informações sem autenticação válida
Íris não deve exibir dados sensíveis caso a verificação de identidade do cliente não esteja concluída com sucesso.

6 - Não executar desbloqueio de cartão
A agente pode permitir o bloqueio por segurança, mas não deve realizar o desbloqueio do cartão.

7 - Não acessar ou expor dados de terceiros
Íris deve limitar as informações exclusivamente ao titular autenticado, sem qualquer exceção.

8 - Não sair do escopo funcional definido
A agente não deve responder ou executar solicitações fora de suas atribuições principais.

9 - Não armazenar informações sensíveis em conversas
Íris não deve solicitar nem reter senhas completas, códigos de segurança ou dados confidenciais desnecessários.

10 - Não fornecer garantias ou promessas
A agente não deve assegurar aprovação de crédito, prazos bancários ou resultados que dependam de análise sistêmica.
"""

#================== CHAMAR LLMs =============

# Function for Groq API
from openai import OpenAI
from google.colab import userdata

def agente_virtual_groq(pergunta: str) -> str:
    client = OpenAI(
        api_key=userdata.get("GROQ_API_KEY"),  # Get your Groq API key from Colab secrets
        base_url="https://api.groq.com/openai/v1"
    )

    resposta = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": pergunta}
        ],
        temperature=0.2
    )

    return resposta.choices[0].message.content

# 🔹 teste Groq (comentado para evitar erros sem a chave)
# print(agente_virtual_groq("Quero consultar meu saldo"))

# Function for OpenAI API (replacing OLLAMA)
def perguntar(pergunta: str, contexto_cliente: str) -> str:
    client = OpenAI(
        api_key=userdata.get("OPENAI_API_KEY")  # Get your OpenAI API key from Colab secrets
    )

    full_prompt = f"{SYSTEM_PROMPT}\n\nInformações do Cliente:\n{contexto_cliente}\n\nPergunta do Cliente: {pergunta}"

    resposta = client.chat.completions.create(
        model="gpt-3.5-turbo",  # You can choose another OpenAI model if preferred
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": full_prompt}
        ],
        temperature=0.2
    )
    return resposta.choices[0].message.content

st.title("Olá, sou seu assistente virtual")

# Initialize BancoDados and AssistenteBancario outside the chat_input loop
banco = BancoDados("/content/") # Corrected path to data files
assistente = AssistenteBancario(banco)

# Set a default client_id for demonstration purposes
# In a real app, this would come from user authentication
if "cliente_id" not in st.session_state:
    st.session_state["cliente_id"] = 1003

cliente_id = st.session_state["cliente_id"]
contexto_cliente = assistente.gerar_contexto_cliente(cliente_id) # Generate context once

if pergunta := st.chat_input("Em que posso ajudar?"):
    st.chat_message("user").write(pergunta)
    with st.spinner("..."):
        st.chat_message("assistant").write(perguntar(pergunta, contexto_cliente)) # Pass contexto_cliente to perguntar

2026-02-19 03:26:10.721 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-19 03:26:10.722 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-19 03:26:10.724 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-19 03:26:10.732 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-19 03:26:10.733 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-19 03:26:10.736 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-19 03:26:10.737 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-19 03:26:10.738 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [25]:
# Download the necessary data files
import requests
import os

# Ensure the /content/sample_data/ directory exists
if not os.path.exists("/content/sample_data/"):
    os.makedirs("/content/sample_data/")

data_files = {
    "saldos_bancarios_ficticios.csv": "https://raw.githubusercontent.com/digitalinnovationone/dio-colab-banking-chatbot/main/data/saldos_bancarios_ficticios.csv",
    "extrato_bancario_ficticio.csv": "https://raw.githubusercontent.com/digitalinnovationone/dio-colab-banking-chatbot/main/data/extrato_bancario_ficticio.csv",
    "limites_cartoes_ficticios.csv": "https://raw.githubusercontent.com/digitalinnovationone/dio-colab-banking-chatbot/main/data/limites_cartoes_ficticios.csv",
    "emprestimos_ficticios.json": "https://raw.githubusercontent.com/digitalinnovationone/dio-colab-banking-chatbot/main/data/emprestimos_ficticios.json",
    "bloqueio_cartoes_ficticios.json": "https://raw.githubusercontent.com/digitalinnovationone/dio-colab-banking-chatbot/main/data/bloqueio_cartoes_ficticios.json",
    "segunda_via_boletos_ficticios.json": "https://raw.githubusercontent.com/digitalinnovationone/dio-colab-banking-chatbot/main/data/segunda_via_boletos_ficticios.json"
}

for filename, url in data_files.items():
    response = requests.get(url)
    if response.status_code == 200:
        with open(f"/content/sample_data/{filename}", "wb") as f:
            f.write(response.content)
        print(f"Downloaded {filename} to /content/sample_data/")
    else:
        print(f"Failed to download {filename}: Status code {response.status_code}")

print("Data download complete.")

Failed to download saldos_bancarios_ficticios.csv: Status code 404
Failed to download extrato_bancario_ficticio.csv: Status code 404
Failed to download limites_cartoes_ficticios.csv: Status code 404
Failed to download emprestimos_ficticios.json: Status code 404
Failed to download bloqueio_cartoes_ficticios.json: Status code 404
Failed to download segunda_via_boletos_ficticios.json: Status code 404
Data download complete.
